In [96]:
from sentence_transformers import SentenceTransformer
import torch
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
import faiss
from transformers import pipeline
from scipy.stats import skew


pd.set_option('display.max_colwidth', 100)
nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [4]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    
)

messages = [
    {"role": "system", "content": "Eres una máquina que reformula consultas sobre historia latinoamericana. A cada consulta responderás con un pasaje de texto corto pero relevante que pueda ser útil para responder a la consulta original. No vas a dar respuestas que incluyan títulos, listas o formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]
messages2 = [
    {"role": "system", "content": "Eres una máquina especializada en reformular consultas sobre historia latinoamericana. A cada consulta, tu tarea es hacerla más precisa y clara, sin extenderte demasiado. Solo responderás con la consulta reformulada, sin incluir títulos, listas o formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]
messages3 = [
    {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]

def get_messages(q):
    return [
        {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
        {"role": "user", "content": q},
    ]

Device set to use cuda:0


In [5]:
q = "Monopolio de Sevilla"

outputs = pipe(
    get_messages(q),
    max_new_tokens=256,
)

new_query = outputs[0]["generated_text"][-1]['content']
new_query

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'El monopolio de Sevilla se refiere a la situación en la que la ciudad de Sevilla, en España, se convirtió en un centro de comercio y comerciante de bienes de lujo, especialmente vino, en el siglo XVII. Esta situación se debió a una combinación de factores, incluyendo la expansión del comercio español en América, la creciente demanda de vino en Europa y la posición geográfica de Sevilla como puerto.\n\nEn 1635, la ciudad de Sevilla se convirtió en el principal centro de comercio de vino en Europa, gracias en parte a la expansión de la industria vinícola en América. La ciudad era un importante centro de exportación de vino, y su reputación como una de las mejores tiendas de vino del mundo se consolidó en ese momento.\n\nEl monopolio de Sevilla se debió en parte a la posición de la ciudad como puerto y el centro de comercio, que la convirtió en un punto de partida para la exportación de vino a todo el mundo. Además, la ciudad tenía una gran cantidad de comerciantes y vendedores de bienes

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [40]:
def divide_document(text, max_words=300):
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]
    fragments = []
    temp = ""

    for sent in sents:
        if len(temp.split()) + len(sent.split()) <= max_words:
            temp += " " + sent
        else:
            fragments.append(temp.strip())
            temp = sent

    if temp:
        fragments.append(temp.strip())

    return fragments


In [7]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-los-Andes-ariqueños-Resistencia-y-conflicto-frente...,"catalogada y caricaturizada bajo el rótulo de indómito pueblo araucano. esta imagen, construida ..."
1,pack\AIBR\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-...,agradecimientos la autora desea agradecer a la profesora luz gabriela arango la lectura del pres...
2,pack\AIBR\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf,"has carried out the biggest employment adjustment in our country, that will finished in 2007, af..."
3,pack\AIBR\2006-Construcción-de-modelos-de-género-a-partir-de-textos-de-la-tradición-oral-en-Extr...,"tampoco queremos que se entienda identidad de género, como identidad del nosotras o identidad de..."
4,pack\AIBR\2006-Contribuciones-feministas-a-problemas-epistemológicos-de-la-disciplina-antropológ...,is irrelevant for others con el que teresa del valle enuncia uno de los apartados de su introduc...
...,...,...
1920,pack\Revista-Española-de-Antropología-Americana\2024-Paridad-y-violencia-política-contra-las-muj...,"cómo citar: burguete cal y mayor, araceli. 2024. paridad y violencia política contra las mujeres..."
1921,pack\Revista-Española-de-Antropología-Americana\2024-Reorientación-de-la-educación-superior-inte...,project exposed some critical questionings. in this paper i expose five of these critical questi...
1922,pack\Revista-Española-de-Antropología-Americana\2024-Una-aproximación-metodológica-a-la-lectura-...,"1. introducción el interés por los quipus, que ha crecido en los últimos años, ha sido acompañad..."
1923,pack\Revista-Española-de-Antropología-Americana\2024-Valdivia-fase-3-transformación-de-las-figur...,1. introducción ¿cómo se produce la diferenciación entre los géneros? ¿cómo surge el conflicto d...


In [40]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

# Carga el modelo y el tokenizer
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [43]:
mi, ma = 1 << 31, 0
for i, r in df.iterrows():
    if type(r["Text"]) != str:
        continue
    n_tokens = len(tokenizer.tokenize(r["Text"]))
    mi = min(mi, n_tokens)
    ma = max(ma, n_tokens)
mi, ma

Token indices sequence length is longer than the specified maximum sequence length for this model (8924 > 512). Running this sequence through the model will result in indexing errors


(65, 44391)

In [42]:
def divide_and_assign_ids(i, text):
    if type(text) != str:
        return [], []

    frags = divide_document(text)
    return frags, [i] * len(frags)

fragments = []
fragments_ids = []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: divide_and_assign_ids(row[0], row[1]["Text"]),
                                df.iterrows()))

    for frags, ids in results:
        if not frags or not ids:
            continue

        fragments.extend(frags)
        fragments_ids.extend(ids)

len(fragments), len(fragments_ids)

KeyboardInterrupt: 

In [ ]:
with open("fragments.pkl", "wb") as f:
    pickle.dump((fragments, fragments_ids), f)

In [28]:
with open("fragments.pkl", "rb") as f:
    fragments, fragments_ids = pickle.load(f)

In [29]:
embeddings = model.encode(fragments, convert_to_tensor=True, device=device)
print(embeddings.shape)

torch.Size([57716, 384])


In [81]:
embeddings_np = embeddings.cpu().numpy()
print(embeddings_np.shape)

# embeddings_normalized = embeddings_np / np.linalg.norm(embeddings_np, axis=1, keepdims=True)
# print(embeddings_normalized.shape)

(73111, 1024)


In [82]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001E5071ED4D0> >

In [21]:
faiss.write_index(index, "index-sentence-transformers.faiss")

In [9]:
index = faiss.read_index("index-sentence-transformers.faiss")

In [83]:
def clean_query(query: str) -> str:
    query = query.replace('"', '').replace("'", '')
    query = query.replace("\n", " ").replace("\r", " ")
    query = query.replace("  ", " ").strip()

    doc = nlp(query)
    sentences = [sent.text.strip() for sent in doc.sents]
    return " ".join([sent for sent in sentences if len(sent) > 0 and sent[-1] == '.']).lower()

def search(query, top_k=10):
    query_embedding = model.encode([clean_query(query)], convert_to_tensor=True, device=device).cpu().numpy()
    distances, indices = index.search(query_embedding, k=top_k*4)

    already_in = set()
    results = []
    for i in indices[0]:
        id = fragments_ids[i]
        if id in already_in:
            continue

        already_in.add(id)
        results.append(id)
        if len(results) == top_k:
            break

    return df.iloc[results]

def skewness(query, top_k=10):
    query_embedding = model.encode([clean_query(query)], convert_to_tensor=True, device=device).cpu().numpy()
    distances, _ = index.search(query_embedding, k=top_k)

    # max and min distances
    print(distances[0].min(), distances[0].max())

    similarities = 1 - distances
    return skew(similarities[0])

In [84]:
search(new_query, top_k=10)

,Path,Text
675,pack\Anuario-de-Estudios-Americanos\2014-La-elaboración-de-la-Ordenanza-de-Montes-de-Marina-de-3...,introducción el mayor punto del comercio de españa depende de arreglar lo casi perdido con sus i...
1736,pack\Revista-Española-de-Antropología-Americana\2015-Interacciones-entre-las-sociedades-y-las-pl...,figura 1: ubicación del área septentrional del río de la plata en el contexto regional. en detal...
1307,pack\Revista-de-Indias\2017-Gamonales-y-alcaldes-poder-institucional-y-parainstitucional-en-la-P...,introducción la presente investigación tiene como principal objetivo recalcar el importante pape...
956,pack\Revista-de-Indias\2002-En-búsqueda-de-un-punto-fijo-para-la-República-El-cesarismo-liberal-...,"and covenants, without the sword, are but words2 tres de las cuatro revoluciones de principios d..."
1116,pack\Revista-de-Indias\2008-Políticas-y-prácticas-migratorias-los-flujos-de-gallegos-y-asturiano...,"ricano de las corrientes del noroeste hispánico, luego del cubano, que mantuvo una notoria y ant..."
435,pack\Anuario-de-Estudios-Americanos\2001-El-oro-de-América-La-contribución-de-los-emigrantes-del...,"ca, y en particular el lerrouxismo. la república del emigrante describe en detalle la cultura po..."
1256,pack\Revista-de-Indias\2014-México-y-las-instituciones-republicanas-en-el-exilio-del-apoyo-del-C...,"normalizaron, el inicio de la guerra civil puso de manifiesto el apoyo inequívoco del gobierno m..."
681,pack\Anuario-de-Estudios-Americanos\2014-Los-arbitristas-y-la-América-portuguesa-15901640.pdf,"introducción a finales de diciembre de 1640, los consejeros de estado de felipe iv se reunieron ..."
1104,pack\Revista-de-Indias\2008-Construyendo-identidades-desde-la-excepcionalidad-mujer-divorciada-y...,"latinoamérica. pero en esta ocasión el nombre es el de maría baldomera fuentes segura, la protag..."
1811,pack\Revista-Española-de-Antropología-Americana\2019-Explorando-las-órbitas-tecnológicas-de-dos-...,"1. introducción treinta y cinco años después del inicio del proyecto nacional tikal, el cual des..."


In [85]:
search(q, top_k=10)

,Path,Text
609,pack\Anuario-de-Estudios-Americanos\2010-Valoración-partición-y-distribución-de-la-tierra-de-los...,"introducción en la nueva granada, a cuya jurisdicción estuvo adscrita mérida hasta 1777, se deno..."
607,pack\Anuario-de-Estudios-Americanos\2010-Sólo-una-virreina-consorte-de-la-Nueva-España-16601664-...,"en la historia colonial del nuevo mundo, el más alto poder residía en los virreyes y las activid..."
862,pack\Anuario-de-Estudios-Americanos\2022-Una-mirada-familiar-a-fray-Juan-de-Zumárraga-ocho-docum...,"en el curso de mis investigaciones en los protocolos de sevilla me he encontrado, no sin cierta ..."
1055,pack\Revista-de-Indias\2006-Empresas-instituciones-y-red-social-la-Compañía-Hispanoamericana-de-...,nos envían a sus familias y comunidades de origen y que son denominados remesas. los retornos de...
1485,pack\Revista-Española-de-Antropología-Americana\2003-Diseño-arquitectónico-patrones-de-ocupación...,1. el sitio de pachacamac pachacamac se encuentra sobre la margen derecha del río lurín y cerca ...
900,pack\Revista-de-Indias\2000-El-crecimiento-económico-de-Cuba-republicana-19021959-Una-revisión-y...,"medida perfecta y tampoco la única posible. sin embargo, su cálculo y el de los demás grandes ag..."
543,pack\Anuario-de-Estudios-Americanos\2007-Militarización-e-identidades-políticas-en-la-revolución...,"ñoles americanos a partir de 1808, este trabajo propone explorar estos procesos de construcción ..."
1314,pack\Revista-de-Indias\2017-La-política-desde-el-campo-iniciativas-locales-y-gobierno-rural-en-t...,planteo introductorio uno de los grandes atractivos de la historia política colonial del siglo x...
478,pack\Anuario-de-Estudios-Americanos\2003-Una-antinomia-protorrenacentista-secreto-de-estado-y-di...,"difícil coexistencia de ambos parámetros, el mayor énfasis de uno u otro —según los intereses es..."
541,pack\Anuario-de-Estudios-Americanos\2007-Las-rutas-del-libro-atlántico-libros-enviados-en-el-nav...,to impreso fue un producto cultural con rasgos novedosos que pronto encontraría su lugar en el m...


In [27]:
skewness(q, top_k=20)

11.059961 14.668214


3.362548656394798

# LANGCHAIN

In [55]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
import faiss
import torch
from langchain.text_splitter import CharacterTextSplitter
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
from transformers import pipeline
from scipy.stats import skew
from langchain_text_splitters import TokenTextSplitter


pd.set_option('display.max_colwidth', 100)
nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [62]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-los-Andes-ariqueños-Resistencia-y-conflicto-frente...,"catalogada y caricaturizada bajo el rótulo de indómito pueblo araucano. esta imagen, construida ..."
1,pack\AIBR\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-...,agradecimientos la autora desea agradecer a la profesora luz gabriela arango la lectura del pres...
2,pack\AIBR\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf,"has carried out the biggest employment adjustment in our country, that will finished in 2007, af..."
3,pack\AIBR\2006-Construcción-de-modelos-de-género-a-partir-de-textos-de-la-tradición-oral-en-Extr...,"tampoco queremos que se entienda identidad de género, como identidad del nosotras o identidad de..."
4,pack\AIBR\2006-Contribuciones-feministas-a-problemas-epistemológicos-de-la-disciplina-antropológ...,is irrelevant for others con el que teresa del valle enuncia uno de los apartados de su introduc...
...,...,...
1920,pack\Revista-Española-de-Antropología-Americana\2024-Paridad-y-violencia-política-contra-las-muj...,"cómo citar: burguete cal y mayor, araceli. 2024. paridad y violencia política contra las mujeres..."
1921,pack\Revista-Española-de-Antropología-Americana\2024-Reorientación-de-la-educación-superior-inte...,project exposed some critical questionings. in this paper i expose five of these critical questi...
1922,pack\Revista-Española-de-Antropología-Americana\2024-Una-aproximación-metodológica-a-la-lectura-...,"1. introducción el interés por los quipus, que ha crecido en los últimos años, ha sido acompañad..."
1923,pack\Revista-Española-de-Antropología-Americana\2024-Valdivia-fase-3-transformación-de-las-figur...,1. introducción ¿cómo se produce la diferenciación entre los géneros? ¿cómo surge el conflicto d...


In [86]:
# model = SentenceTransformer("sentence-transformers/static-similarity-mrl-multilingual-v1", trust_remote_code=True)
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", trust_remote_code=True)

In [90]:
text_splitter = TokenTextSplitter(chunk_size=300, chunk_overlap=50)

In [91]:
chunks, chunks_ids = [], []
for i, r in df.iterrows():
    if type(r["Text"]) != str:
        continue

    new_chunks = text_splitter.split_text(r["Text"])
    chunks.extend(new_chunks)
    chunks_ids.extend([i] * len(new_chunks))

len(chunks)

134295

In [94]:
# embeddings = model.encode(chunks, convert_to_tensor=True, device=device, prompt_name="passage")
embeddings = model.encode(chunks, convert_to_tensor=True, device=device)
embeddings.shape

KeyboardInterrupt: 

In [95]:
embeddings_np = embeddings.cpu().numpy()
embeddings_np.shape

(100, 384)

In [ ]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

faiss.write_index(index, "index-nomic-ai.faiss")

faiss_store = FAISS(index)

In [ ]:
from sentence_transformers import SentenceTransformer

sentences = ["Hello!", "¡Hola!"]
embeddings = model.encode(sentences, prompt_name="passage")
print(embeddings.shape)
# (2, 768)

similarity = model.similarity(embeddings[0], embeddings[1])
print(similarity)
# tensor([[0.9118]]) 

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'